# 02: Create functions for getting forecasts

In [1]:
# Renewable Forecast Payload
start_date = "20220805"
end_date = "20220806"
market_run_id = "DAM"

#market_run_id = "SYS_FCST_DA_MW"
#data_item = "SLD_REN_FCST" # This is just renewables - solar + wind (what is forecasted and also actual)
#data_item = "ISO_TOT_GEN_MW" # This is the generation (there are also forecasts for next day)

data_item = "ENE_SLRS"
#data_item = "SLD_FCST" # This is the demand forecasted (so how much customers need)

renewable_url = f"http://oasis.caiso.com/oasisapi/SingleZip?queryname={data_item}&" \
              f"startdatetime={start_date}T07:00-0000&" \
              f"enddatetime={end_date}T07:00-0000&version=1&"\
              f"market_run_id={market_run_id}"


renewable_url

'http://oasis.caiso.com/oasisapi/SingleZip?queryname=ENE_SLRS&startdatetime=20220805T07:00-0000&enddatetime=20220806T07:00-0000&version=1&market_run_id=DAM'

In [2]:
#os.chdir('..')
from emissions_web.helpers.caiso_helpers import *

In [13]:
all_gen = get_oasis_forecast(start_date, end_date, market_run_id, data_item)

In [14]:
all_gen.head()

,value,start_date,end_date,resource_name,data_item
0,1735,2022-08-05T10:00:00-00:00,2022-08-05T11:00:00-00:00,Caiso_Totals,ISO_TOT_EXP_MW
1,1727,2022-08-05T12:00:00-00:00,2022-08-05T13:00:00-00:00,Caiso_Totals,ISO_TOT_EXP_MW
2,3329,2022-08-05T21:00:00-00:00,2022-08-05T22:00:00-00:00,Caiso_Totals,ISO_TOT_EXP_MW
3,2171,2022-08-05T14:00:00-00:00,2022-08-05T15:00:00-00:00,Caiso_Totals,ISO_TOT_EXP_MW
4,1718,2022-08-05T11:00:00-00:00,2022-08-05T12:00:00-00:00,Caiso_Totals,ISO_TOT_EXP_MW


In [6]:
solar_wind = get_oasis_forecast(start_date, end_date, market_run_id, 'SLD_REN_FCST')

In [7]:
solar_wind.head()

,value,start_date,end_date,trading_hub,ren_type,data_item
0,0,2022-08-05T07:00:00-00:00,2022-08-05T08:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW
1,0,2022-08-05T08:00:00-00:00,2022-08-05T09:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW
2,0,2022-08-05T09:00:00-00:00,2022-08-05T10:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW
3,0,2022-08-05T10:00:00-00:00,2022-08-05T11:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW
4,0,2022-08-05T11:00:00-00:00,2022-08-05T12:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW


In [11]:
pd.unique(solar_wind['trading_hub'])

array(['NP15', 'SP15', 'ZP26'], dtype=object)

In [12]:
pd.unique(solar_wind['data_item'])

array(['RENEW_FCST_DA_MW'], dtype=object)

In [8]:
other_solar = get_oasis_forecast(start_date, end_date, market_run_id, 'SLD_FCST')

In [9]:
other_solar.head()

,value,start_date,end_date,resource_name,data_item
0,1221.85,2022-08-06T04:00:00-00:00,2022-08-06T05:00:00-00:00,AVA,SYS_FCST_DA_MW
1,1017.22,2022-08-05T12:00:00-00:00,2022-08-05T13:00:00-00:00,AVA,SYS_FCST_DA_MW
2,1009.42,2022-08-05T07:00:00-00:00,2022-08-05T08:00:00-00:00,AVA,SYS_FCST_DA_MW
3,1329.54,2022-08-05T18:00:00-00:00,2022-08-05T19:00:00-00:00,AVA,SYS_FCST_DA_MW
4,1292.89,2022-08-05T17:00:00-00:00,2022-08-05T18:00:00-00:00,AVA,SYS_FCST_DA_MW


In [10]:
pd.unique(other_solar['resource_name'])

array(['AVA', 'AZPS', 'BANC', 'BANCMID', 'BANCRDNG', 'BANCRSVL',
       'BANCSMUD', 'BANCWASN', 'BPAT', 'CA ISO-TAC', 'IPCO', 'LADWP',
       'MWD-TAC', 'NEVP', 'NWMT', 'PACE', 'PACW', 'PGE', 'PGE-TAC', 'PNM',
       'PSEI', 'SCE-TAC', 'SCL', 'SDGE-TAC', 'SRP', 'TEPC', 'TIDC',
       'TPWR', 'VEA-TAC'], dtype=object)

## Cleaning Data

- have to convert data types to dates/numbers

In [15]:
all_gen.describe()

,value,start_date,end_date,resource_name,data_item
count,456,456,456,456,456
unique,411,24,24,6,8
top,0,2022-08-05T10:00:00-00:00,2022-08-05T11:00:00-00:00,Caiso_Totals,TOT_LOAD_MW
freq,21,19,19,96,120


In [16]:
solar_wind.describe()

,value,start_date,end_date,trading_hub,ren_type,data_item
count,120,120,120,120,120,120
unique,95,24,24,3,2,1
top,0,2022-08-05T07:00:00-00:00,2022-08-05T08:00:00-00:00,NP15,Solar,RENEW_FCST_DA_MW
freq,26,5,5,48,72,120


In [17]:
pd.unique(all_gen['data_item'])

array(['ISO_TOT_EXP_MW', 'ISO_TOT_GEN_MW', 'ISO_TOT_IMP_MW',
       'ISO_TOT_LOAD_MW', 'TOT_LOAD_MW', 'TOT_EXP_MW', 'TOT_GEN_MW',
       'TOT_IMP_MW'], dtype=object)

In [18]:
solar_wind_actual = get_oasis_forecast('20220819', '20220820', 'ACTUAL', 'SLD_REN_FCST')

In [26]:
solar_wind_actual

,value,start_date,end_date,trading_hub,ren_type,data_item
0,-4.48703,2022-08-19T07:00:00-00:00,2022-08-19T08:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
1,1571.37577,2022-08-19T15:00:00-00:00,2022-08-19T16:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
2,1716.34932,2022-08-19T23:00:00-00:00,2022-08-20T00:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
3,1827.94496,2022-08-19T22:00:00-00:00,2022-08-19T23:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
4,1770.37962,2022-08-19T21:00:00-00:00,2022-08-19T22:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
...,...,...,...,...,...,...
80,-5.97101,2022-08-19T09:00:00-00:00,2022-08-19T10:00:00-00:00,ZP26,Solar,RENEW_FCST_ACT_MW
81,1738.42269,2022-08-19T16:00:00-00:00,2022-08-19T17:00:00-00:00,ZP26,Solar,RENEW_FCST_ACT_MW
82,-5.91156,2022-08-19T08:00:00-00:00,2022-08-19T09:00:00-00:00,ZP26,Solar,RENEW_FCST_ACT_MW
83,1969.09905,2022-08-19T19:00:00-00:00,2022-08-19T20:00:00-00:00,ZP26,Solar,RENEW_FCST_ACT_MW


In [20]:
solar_wind_actual.describe()

,value,start_date,end_date,trading_hub,ren_type,data_item
count,85,85,85,85,85,85
unique,85,17,17,3,2,1
top,-4.48703,2022-08-19T07:00:00-00:00,2022-08-19T08:00:00-00:00,NP15,Solar,RENEW_FCST_ACT_MW
freq,1,5,5,34,51,85


In [22]:
solar_wind_actual_agg = solar_wind_actual.groupby(['start_date', 'ren_type']).agg({'value': 'sum'})

In [23]:
solar_wind_actual_agg

value
start_date                ren_type                                
2022-08-19T07:00:00-00:00 Solar            -4.48703-32.96085-5.882
                          Wind                1108.358132745.33436
2022-08-19T08:00:00-00:00 Solar          -4.50568-32.41668-5.91156
                          Wind                1060.280942703.07411
2022-08-19T09:00:00-00:00 Solar           -4.50279-88.0799-5.97101
                          Wind                1076.481032562.83713
2022-08-19T10:00:00-00:00 Solar          -4.52271-29.66621-5.93407
                          Wind                1078.561192322.53035
2022-08-19T11:00:00-00:00 Solar          -4.47701-37.00287-5.99957
                          Wind                1070.838081845.77051
2022-08-19T12:00:00-00:00 Solar           -4.44602-35.8234-6.05599
                          Wind                1062.289541263.92849
2022-08-19T13:00:00-00:00 Solar           36.73882274.667327.30328
                          Wind                  975.92812654.86073
2022-08-19T14:00:00-00:00 Solar       768.900872914.07731603.99483
                          Wind                  898.82525326.67489
2022-08-19T15:00:00-00:00 Solar      1571.375775999.84071379.21321
                          Wind                  789.42509187.34042
2022-08-19T16:00:00-00:00 Solar     1763.820777291.597241738.42269
                          Wind                  699.71347154.03258
2022-08-19T17:00:00-00:00 Solar     1777.822027727.141541879.05729
                          Wind                  499.58445116.32304
2022-08-19T18:00:00-00:00 Solar       1769.2438163.160451928.60005
                          Wind                   364.9290395.22789
2022-08-19T19:00:00-00:00 Solar     1777.943518515.212081969.09905
                          Wind                  357.92491168.50497
2022-08-19T20:00:00-00:00 Solar     1770.837478777.564661942.21534
                          Wind                  423.07801505.75313
2022-08-19T21:00:00-00:00 Solar     1770.379628517.993091927.32972
                          Wind                  505.52672975.18882
2022-08-19T22:00:00-00:00 Solar     1827.944968180.985891862.45116
                          Wind                  564.632091521.4663
2022-08-19T23:00:00-00:00 Solar     1716.349327493.938261728.46063
                          Wind                 611.048641836.70256

In [24]:
supply_trend = get_supply_trend_today()

In [25]:
supply_trend

,Solar,Wind,Geothermal,Biomass,Biogas,Small hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other,DateTime
0,-41,3948,923,386,212,189,14,2261,11776,2504,-2,7451,0,2022-08-19 00:00:00
1,-43,3926,923,384,212,188,14,2261,11952,2390,517,7025,0,2022-08-19 00:05:00
2,-43,3912,922,383,211,189,13,2260,11826,2372,637,6875,0,2022-08-19 00:10:00
3,-44,3888,923,376,211,188,14,2261,11743,2457,421,6969,0,2022-08-19 00:15:00
4,-43,3879,922,368,211,187,14,2261,11616,2370,265,7236,0,2022-08-19 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,9703,3177,854,398,207,226,14,2266,15848,3107,667,4793,0,2022-08-19 17:35:00
212,9392,3151,853,399,207,228,15,2266,15971,3118,628,5091,0,2022-08-19 17:40:00
213,8996,3180,854,399,207,229,14,2266,16120,3131,634,5384,0,2022-08-19 17:45:00
214,8519,3179,855,399,207,228,14,2266,16377,3150,825,5413,0,2022-08-19 17:50:00
